In [99]:
import pandas as pd
from pathlib import Path

APP_DIR = Path("/run/media/mix/64B2AAFCB2AAD1BA/APP/")

In [100]:
# glob all xlsx files
xlsx_files = list(APP_DIR.glob("**/*.xlsx"))

# read all xlsx files
xlsx_data: dict[Path, pd.DataFrame] = {}
for file in xlsx_files:
    xlsx_data[file] = pd.read_excel(file)

pd.concat(xlsx_data.values())

/home/mix/Documents/MalClass/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/mix/Documents/MalClass/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/mix/Documents/MalClass/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/mix/Documents/MalClass/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,诈骗类型,APP名称,APP包名,apk文件名
0,投资理财,共富工程,uni.tlfy,1719354472896
1,投资理财,慈联,com.zjasmdxa.fiyyrs,1719372932233
2,投资理财,国能APP,com.ekatyhlo.mirpwb,1719402160462
3,投资理财,共富工程,uni.xqrh,1719347300879
4,贷款诈骗,还呗,com.example.byaf.byaf,1719398212372
...,...,...,...,...
193,投资理财,强国复兴,uni.khwm,1719713801202
194,投资理财,慈联,com.dwrcsaht.hzcelz,1719701530760
195,投资理财,强国复兴,uni.xhno,1719698243724
196,投资理财,共富工程,uni.bckx,1719699541723


In [101]:
corresponding_apk = {
    pkg_name: next(file.parent.glob(f"**/{filename}.apk"))
    for file, dataframe in xlsx_data.items()
    for _, _, pkg_name, filename in dataframe.values
}

In [102]:
import libarchive
import mimetypes
import re
from tqdm.notebook import tqdm


IPV4_PORT_REGEX = re.compile(
    rb"(?P<ip>(?:(?:25[0-5]|(2[0-4]|1\d|[1-9]|)\d)\.?\b){4})"  # ipv4
    rb"(?::(?P<port>6553[0-5]|655[0-2]\d|65[0-4]\d{2}|6[0-4]\d{3}|[1-5]\d{4}|\d{1,4}))?"  # port
)

with open("./public_suffix_list.dat", "rb") as f:
    suffixes = sorted(
        stripped
        for line in f.readlines()
        if (stripped := line.strip())
        and not stripped.startswith(b"//")
        and stripped.isascii()
    )
    PUBLIC_SUFFIXES_REGEX = re.compile(
        rb"(?:"
        + rb"|".join(re.escape(suffix) for suffix in suffixes)
        + rb")(?![a-zA-Z.()])"
    )


DOMAIN_REGEX = re.compile(
    rb"(?<![a-zA-Z.()])(?:(?!-)[A-Za-z0-9-]{2,63}(?<!-)\.){1,}"
    + PUBLIC_SUFFIXES_REGEX.pattern
)
print(DOMAIN_REGEX.pattern)
HTTP_URL_REGEX = re.compile(rb"https?://[A-Za-z0-9_.\-~:/?#\[\]@!$&'()*+,;=]+\/?")

new_df = pd.DataFrame(
    columns=["pkg_name", "hosts", "urls"],
)

for pkg_name, apk_path in tqdm(corresponding_apk.items(), total=len(corresponding_apk)):
    hosts = set[bytes]()
    urls = set[bytes]()
    with (
        libarchive.file_reader(str(apk_path)) as archive,
        tqdm(
            leave=False,
            desc=f"Processing {pkg_name}",
        ) as progress_bar,
    ):
        for entry in archive:
            if re.match(r"^(?:res|assets)", entry.pathname):
                continue
            mimetype, _ = mimetypes.guess_type(entry.pathname)
            if mimetype and re.match(r"^(?:image|audio|video)", mimetype):
                continue
            try:
                entry_file = b"".join(entry.get_blocks())
            except Exception as e:
                tqdm.write(repr(e))
                continue
            hosts.update(match[0] for match in IPV4_PORT_REGEX.finditer(entry_file))
            hosts.update(match[0] for match in DOMAIN_REGEX.finditer(entry_file))
            progress_bar.update(1)
            urls.update(match[0] for match in HTTP_URL_REGEX.finditer(entry_file))
    new_df.loc[len(new_df)] = [pkg_name, sorted(hosts), sorted(urls)]

new_df

b'(?<![a-zA-Z.()])(?:(?!-)[A-Za-z0-9-]{2,63}(?<!-)\\.){1,}(?:!city\\.kawasaki\\.jp|!city\\.kitakyushu\\.jp|!city\\.kobe\\.jp|!city\\.nagoya\\.jp|!city\\.sapporo\\.jp|!city\\.sendai\\.jp|!city\\.yokohama\\.jp|!ignored\\.sub\\.wc\\.psl\\.hrsn\\.dev|!ignored\\.wc\\.psl\\.hrsn\\.dev|!www\\.ck|\\*\\.001\\.test\\.code\\-builder\\-stg\\.platform\\.salesforce\\.com|\\*\\.0e\\.vc|\\*\\.0emm\\.com|\\*\\.advisor\\.ws|\\*\\.af\\-south\\-1\\.airflow\\.amazonaws\\.com|\\*\\.alces\\.network|\\*\\.ap\\-east\\-1\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-northeast\\-1\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-northeast\\-2\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-northeast\\-3\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-south\\-1\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-south\\-2\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-southeast\\-1\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-southeast\\-2\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-southeast\\-3\\.airflow\\.amazonaws\\.com|\\*\\.ap\\-southeast\\-4\\.airflow\\.amazonaws\\.com|\\

  0%|          | 0/783 [00:00<?, ?it/s]

Processing uni.tlfy: 0it [00:00, ?it/s]

Processing com.zjasmdxa.fiyyrs: 0it [00:00, ?it/s]

Processing com.ekatyhlo.mirpwb: 0it [00:00, ?it/s]

...

Processing com.dwrcsaht.hzcelz: 0it [00:00, ?it/s]

Processing uni.xhno: 0it [00:00, ?it/s]

Processing uni.bckx: 0it [00:00, ?it/s]

Processing chdhcdbc.ecdddgad.bbegehce: 0it [00:00, ?it/s]

,pkg_name,hosts,urls
0,uni.tlfy,"[b'0.0.0.0', b'000.pn', b'005.pn', b'012345678...","[b'http://lame.sf.net', b'http://localhost', b..."
1,com.zjasmdxa.fiyyrs,"[b'000.pn', b'005.pn', b'0150898.directory', b...","[b'http://16.163.94.194:8011', b'http://dashif..."
2,com.ekatyhlo.mirpwb,"[b'000.pn', b'005.pn', b'0150898.directory', b...","[b'http://103.135.32.219:8011', b'http://dashi..."
3,uni.xqrh,"[b'0.0.0.0', b'000.pn', b'005.pn', b'012345678...","[b'http://lame.sf.net', b'http://localhost', b..."
4,com.example.byaf.byaf,"[b'000.pn', b'005.pn', b'AbsActionBarView.java...",[b'http://schemas.android.com/apk/res/android']
...,...,...,...
778,uni.khwm,"[b'0.0.0.0', b'000.pn', b'005.pn', b'012345678...","[b'http://lame.sf.net', b'http://localhost', b..."
779,com.dwrcsaht.hzcelz,"[b'000.pn', b'005.pn', b'0150898.directory', b...","[b'http://43.199.23.157:8011', b'http://dashif..."
780,uni.xhno,"[b'0.0.0.0', b'000.pn', b'005.pn', b'012345678...","[b'http://lame.sf.net', b'http://localhost', b..."
781,uni.bckx,"[b'0.0.0.0', b'000.pn', b'005.pn', b'012345678...","[b'http://lame.sf.net', b'http://localhost', b..."


In [106]:
new_df["urls"], new_df["hosts"]

new_df.to_csv(APP_DIR / "apps.csv", index=False)